In [ ]:
from time import time
import pandas as pd
import configparser 
import boto3
import time
import json
#import psycopg2 



In [ ]:
# DWH_ENDPOINT = config.get("CLUSTER", "HOST")[1:-1] 
# DWH_ROLE_ARN = config.get("IAM_ROLE", "ARN")[1:-1] 



**1ere fonction def loadParams()**: pour initialiser les parametres et les ressources AWS, pas de valeurs d'entree, ne renvoie rien 

In [ ]:
import configparser 
config = configparser.ConfigParser() # creer le fichier de configuaration en memoire
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("IAM_ROLE", "DWH_IAM_ROLE_NAME")


param = pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

print(param)

print(KEY) # a effacer en final

import boto3

ec2 = boto3.resource('ec2', 
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name="us-west-2")

s3 = boto3.resource('s3', 
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET,
                    region_name="us-west-2")

iam = boto3.client('iam',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET,
                  region_name="us-west-2"
                  )

redshift = boto3.client('redshift',
                        region_name="us-west-2",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                       )



**2eme fonction def createRole(iam, DWH_IAM_ROLE_NAME)** : creer IAM role et la policy, qui permets aux clusters de redshift de joindre les srvice aws, valeur d'entree: iam et DWH_IAM_ROLE_NAME, sortie: roleArn

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating a new IAM Role...") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)

print("Attaching Policy...")
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']


print("Get the IAM role ARN...")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print('  ARN ---> {} :'.format(roleArn))

**3eme fonction**
: create_cluster. pour creer le cluster. params:redshift, roleARN DWH_CLUSTER_TYPE, DWH_NODE_TYPE, DWH_NUM_NODES, DWH_DB, DWH_CLUSTER_IDENTIFIER, DWH_DB_USER,DWH_DB_PASSWORD 
return : 

In [ ]:
print('--->> Create cluster beginning... <<---')
try:
    response = redshift.create_cluster(        
        # parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # parameters for identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,       
        
        # parameter for role (to allow s3 access)
        IamRoles=[roleArn]
         
    )
except Exception as e:
    print(e)
    


**4eme fonction** def prettyRedshiftProps(props): pour voir l'etat de la creation du cluster

In [ ]:
    print('--->> Create cluster processing... <<---')
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

In [ ]:
# **my path**
# load parametres
def loadParams()
# creer arnRole
def createRole(iam, DWH_IAM_ROLE_NAME)
# create le cluster
def createCluster(redshift, roleARN DWH_CLUSTER_TYPE, DWH_NODE_TYPE, DWH_NUM_NODES, DWH_DB, DWH_CLUSTER_IDENTIFIER, DWH_DB_USER,DWH_DB_PASSWORD)
# afficher l'etat du cluster
def prettyRedshiftProps(props)
# faire patienter

# qd etat available, afficher l'etat du cluster
def prettyRedshiftProps(props)
# recupere les parametres du cluster
def getParamCluster()
# update DWH_ROLE_ARN et DWH_ENDPOINT du nouveayu cluster

# se connecter au cluster